In [42]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [43]:
f = open("news.txt", "r",encoding="utf8") 
text=f.read()

In [44]:
#text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."
#text="Those Who Are Resilient Stay In The Game Longer On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow. — Friedrich NietzscheChallenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed"
nlp = spacy.load("en_core_web_sm")

In [45]:

import pytextrank

tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

In [46]:

for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)

0.2216     1  shielded twisted pair
[shielded twisted pair]
0.1451     2  pair.the unshielded twisted pair
[pair.the Unshielded twisted pair, security.the Unshielded twisted pair]
0.1412     1  the unshielded twisted pair cable
[The Unshielded twisted pair cable]
0.1401     1  less data
[less data]
0.1360     1  a unshielded twisted pair
[a Unshielded twisted pair]
0.1356     1  less cost
[less cost]
0.1326     1  eia.the unshielded twisted pair
[EIA.the Unshielded twisted pair]
0.1183     1  low datarate
[low datarate]
0.1079     1  present.it
[present.it]
0.1067     1  unshielded
[Unshielded]
0.0719     1  no metallic shield
[no metallic shield]
0.0658     1  10mbps
[10Mbps]


In [47]:
sent_bounds = [ [s.start, s.end, set([])] for s in doc.sents ]
sent_bounds

[[0, 18, set()], [18, 29, set()], [29, 57, set()]]

In [48]:
limit_phrases = 4

phrase_id = 0
unit_vector = []

for p in doc._.phrases:
    print(phrase_id, p.text, p.rank)
    
    unit_vector.append(p.rank)
    
    for chunk in p.chunks:
        print(" ", chunk.start, chunk.end)
        
        for sent_start, sent_end, sent_vector in sent_bounds:
            if chunk.start >= sent_start and chunk.start <= sent_end:
                print(" ", sent_start, chunk.start, chunk.end, sent_end)
                sent_vector.add(phrase_id)
                break

    phrase_id += 1

    if phrase_id == limit_phrases:
        break

0 shielded twisted pair 0.22162577144974907
  54 57
  29 54 57 57
1 pair.the unshielded twisted pair 0.14511410206251552
  18 22
  0 18 22 18
  37 41
  29 37 41 57
2 the unshielded twisted pair cable 0.14122834046313237
  29 34
  18 29 34 29
3 less data 0.14007108490782016
  35 37
  29 35 37 57


In [49]:
sent_bounds

[[0, 18, {1}], [18, 29, {2}], [29, 57, {0, 1, 3}]]

In [50]:

for sent in doc.sents:
    print(sent)

In a Unshielded twisted pair there is no metallic shield is present.it is less reliable than shielded twisted
pair.the Unshielded twisted pair has low datarate near about 10Mbps

The Unshielded twisted pair cable has less data security.the Unshielded twisted pair was invented by EIA.the Unshielded twisted pair has less cost as compare to shielded twisted pair


In [51]:
unit_vector

[0.22162577144974907,
 0.14511410206251552,
 0.14122834046313237,
 0.14007108490782016]

In [52]:
sum_ranks = sum(unit_vector)
unit_vector = [ rank/sum_ranks for rank in unit_vector ]

unit_vector

[0.34199433866384726,
 0.2239279350999151,
 0.21793175306885681,
 0.21614597316738088]

In [53]:
from math import sqrt

sent_rank = {}
sent_id = 0

for sent_start, sent_end, sent_vector in sent_bounds:
    print(sent_vector)
    sum_sq = 0.0
    
    for phrase_id in range(len(unit_vector)):
        print(phrase_id, unit_vector[phrase_id])
        
        if phrase_id not in sent_vector:
            sum_sq += unit_vector[phrase_id]**2.0

    sent_rank[sent_id] = sqrt(sum_sq)
    sent_id += 1

print(sent_rank)

{1}
0 0.34199433866384726
1 0.2239279350999151
2 0.21793175306885681
3 0.21614597316738088
{2}
0 0.34199433866384726
1 0.2239279350999151
2 0.21793175306885681
3 0.21614597316738088
{0, 1, 3}
0 0.34199433866384726
1 0.2239279350999151
2 0.21793175306885681
3 0.21614597316738088
{0: 0.4595361339331887, 1: 0.46240991502422196, 2: 0.21793175306885681}


In [54]:
from operator import itemgetter

sorted(sent_rank.items(), key=itemgetter(1))

[(2, 0.21793175306885681), (0, 0.4595361339331887), (1, 0.46240991502422196)]

In [55]:
limit_sentences = 2

sent_text = {}
sent_id = 0

for sent in doc.sents:
    sent_text[sent_id] = sent.text
    sent_id += 1

num_sent = 0

for sent_id, rank in sorted(sent_rank.items(), key=itemgetter(1)):
    print(sent_id, sent_text[sent_id])
    num_sent += 1
    
    if num_sent == limit_sentences:
        break

2 The Unshielded twisted pair cable has less data security.the Unshielded twisted pair was invented by EIA.the Unshielded twisted pair has less cost as compare to shielded twisted pair
0 In a Unshielded twisted pair there is no metallic shield is present.it is less reliable than shielded twisted
